In [1]:
!pip install transformers datasets accelerate -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 30.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 

# 텍스트 분류

네이버 영화 리뷰 감성 분류: Naver sentiment movie corpus (NSMC)

데이터셋
- train: 150K
- test: 50K

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, pipeline
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import os
import gc
from tqdm.notebook import trange, tqdm

In [3]:
dataset_id = "e9t/nsmc"
model_id = "klue/roberta-small"

# 데이터셋 로드

In [4]:
ds = load_dataset(dataset_id, trust_remote_code=True)

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


# 토크나이저 설정

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

토크나이징 함수 정의

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["document"], truncation=True, padding=True)

데이터셋에 토크나이징 적용

In [8]:
tokenized_datasets = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

구글 코랩 T4에서 실습시 오래 걸리기 때문에 상위 10%씩의 데이터만 사용

In [9]:
tokenized_datasets['train'] = tokenized_datasets['train'].select(range(len(tokenized_datasets['train'])//10))
tokenized_datasets['test'] = tokenized_datasets['test'].select(range(len(tokenized_datasets['test']) // 10))

# DataCollator 설정

In [10]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## 확인

In [ ]:
ts = tokenized_datasets['train']
print(ts.features)

In [ ]:
samples = [{k: ts[i][k] for k in ts.features if k in ['input_ids', 'token_type_ids', 'attention_mask', 'label']} for i in range(4)]

In [ ]:
# DataCollator 적용
batch = data_collator(samples)

In [ ]:
print(batch)

# 모델 로드

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

커스텀 레이블 추가

In [12]:
# 레이블 정의
id2label = {0: "부정", 1: "긍정"}
label2id = {"부정": 0, "긍정": 1}

# 설정 로드 및 레이블 지정
config = AutoConfig.from_pretrained(model_id,
                                    num_labels=2,
                                    id2label=id2label,
                                    label2id=label2id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/545 [00:00<?, ?B/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 평가 메트릭 정의

In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Trainer 설정

In [15]:
os.environ["WANDB_PROJECT"]="HF_workshop"

wandb 계정이 있다면 셀 실행 / 없다면 패스

In [17]:
from google.colab import userdata
import wandb

wandb_api_key = userdata.get('WANDB_API_KEY')
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to Weights & Biases")
else:
    print("WANDB_API_KEY not found in Colab secrets")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases


In [18]:

training_args = TrainingArguments(
    output_dir="./model-by-trainer",
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_ratio=0.1, # 전체 스텝의 10%는 웜업에 사용
    learning_rate=1e-4, # 학습률
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb", # wandb 계정이 없다면 "none" 으로 설정
    run_name="nsmc-text-classfication",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 훈련

## Trainer 활용 훈련

참고: 샘플링하지 않고 전체 데이터 훈련시
- T4 에서는 에포크 당 25분 걸림
- RTX4090에서는 에포크 당 3분40초 걸림

In [19]:
trainer.train()

wandb: Currently logged in as: jangmin-o (ozlab). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.326800,0.332454,0.852800,0.860447,0.824491,0.899683
2,0.222600,0.314692,0.872400,0.874607,0.867108,0.882236
3,0.132800,0.400899,0.873600,0.876127,0.866279,0.886201


TrainOutput(global_step=705, training_loss=0.2611879218554666, metrics={'train_runtime': 515.5299, 'train_samples_per_second': 87.289, 'train_steps_per_second': 1.368, 'total_flos': 1478118839675040.0, 'train_loss': 0.2611879218554666, 'epoch': 3.0})

## 커스텀 훈련 루프

In [20]:
# 메모리 해제 (완벽하게 해제되지는 않음)
del model
del trainer
torch.cuda.empty_cache()  # GPU 메모리 정리
gc.collect()  # 가비지 컬렉션 실행

1413

In [21]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)
model.to(device)

compact_tokenized_datasets = tokenized_datasets.remove_columns(['id', 'document'])

train_dataloader = DataLoader(compact_tokenized_datasets["train"], batch_size=32, shuffle=True, collate_fn=data_collator)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1}/{num_epochs} completed")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 1/3 completed


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 2/3 completed


  0%|          | 0/469 [00:00<?, ?it/s]

Epoch 3/3 completed


## 커스텀 훈련 검증 평가

In [22]:
model.eval()
eval_dataloader = DataLoader(compact_tokenized_datasets["test"], batch_size=64, collate_fn=data_collator)

total_preds = []
total_labels = []

with torch.no_grad():
    for batch in tqdm(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=-1)
        total_preds.extend(preds.cpu().numpy())
        total_labels.extend(batch["labels"].cpu().numpy())

accuracy = accuracy_score(total_labels, total_preds)
precision, recall, f1, _ = precision_recall_fscore_support(total_labels, total_preds, average='binary')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

  0%|          | 0/79 [00:00<?, ?it/s]

Accuracy: 0.8678
F1 Score: 0.8671890697207152
Precision: 0.8790224032586558
Recall: 0.8556701030927835


# 파이프라인을 통한 테스트

In [23]:
classification_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

In [24]:
classification_pipeline("모처럼 눈물 흘렸습니다.")

[{'label': '긍정', 'score': 0.90943843126297}]